In [ ]:
import os
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
import numpy as np
from matplotlib.patheffects import withStroke
from shapely.geometry import Point
from matplotlib.lines import Line2D

import matplotlib
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Nirmala UI']

# --------------------------------------------------
# HELPER: REMOVE WHITE BACKGROUND BUT KEEP BLACK LINES
# --------------------------------------------------
def simple_black_outline(img, white_threshold=200):
    if img.ndim == 2:
        img = np.stack([img, img, img, np.ones_like(img)], axis=-1)
    elif img.shape[2] == 3:
        alpha = np.ones((img.shape[0], img.shape[1], 1))
        img = np.concatenate([img, alpha], axis=2)

    if img.max() <= 1:
        img = (img * 255).astype(np.uint8)

    result = np.zeros((img.shape[0], img.shape[1], 4), dtype=np.uint8)
    r, g, b, a = img[..., 0], img[..., 1], img[..., 2], img[..., 3]
    luminance = 0.299 * r + 0.587 * g + 0.114 * b
    mask = luminance < white_threshold

    result[mask, :3] = 0
    result[mask, 3] = 255
    result[~mask, 3] = 0
    return result

# --------------------------------------------------
# NORTH ARROW: NEPALI PAGODA STYLE (INSIDE AXES)
# --------------------------------------------------
def draw_nepali_pagoda_north_arrow(ax, x=0.92, y=0.95, size=0.07, color="#1b1b1e"):
    from matplotlib.patches import Polygon, Rectangle

    # Pillar
    pillar_width = size * 0.12
    pillar_height = size * 0.55
    pillar = Rectangle(
        (x - pillar_width / 2, y),
        pillar_width,
        pillar_height,
        transform=ax.transAxes,
        facecolor=color,
        edgecolor="none",
        zorder=300
    )
    ax.add_patch(pillar)

    # Roof layers
    roof_heights = [0.10, 0.08, 0.06]
    roof_widths = [0.55, 0.42, 0.30]
    current_y = y + pillar_height * 0.55
    for h, w in zip(roof_heights, roof_widths):
        roof = Polygon(
            [
                (x - size * w / 2, current_y),
                (x + size * w / 2, current_y),
                (x, current_y + size * h),
            ],
            closed=True,
            transform=ax.transAxes,
            facecolor=color,
            edgecolor="none",
            zorder=301
        )
        ax.add_patch(roof)
        current_y += size * h * 0.85

    # Finial
    finial = Polygon(
        [
            (x - size * 0.05, current_y),
            (x + size * 0.05, current_y),
            (x, current_y + size * 0.18),
        ],
        closed=True,
        transform=ax.transAxes,
        facecolor=color,
        edgecolor="none",
        zorder=302
    )
    ax.add_patch(finial)

    # N label
    ax.text(
        x,
        current_y + size * 0.22,
        "N",
        transform=ax.transAxes,
        ha="center",
        va="bottom",
        fontsize=12,
        fontweight="bold",
        color=color,
        zorder=303
    )

# --------------------------------------------------
# COLOR CONFIG
# --------------------------------------------------
COLORS = {
    "background": "#f5f1e8",
    "water": "#a8d0e6",
    "government_land": "#4a4e69",
    "admin_buildings": "#1b1b1e",
    "primary_roads": "#000000",
    "secondary_roads": "#222222",
    "tertiary_roads": "#444444",
    "residential_roads": "#666666",
    "park_green": "#c9e4ca",
    "text_primary": "#1b1b1e",
    "text_secondary": "#555555",
}

ROAD_HIERARCHY = {
    "primary": {"types": ["motorway", "trunk", "primary"], "width": 2.2, "color": COLORS["primary_roads"]},
    "secondary": {"types": ["secondary"], "width": 1.4, "color": COLORS["secondary_roads"]},
    "tertiary": {"types": ["tertiary"], "width": 0.9, "color": COLORS["tertiary_roads"]},
    "residential": {"types": ["residential", "living_street", "unclassified"], "width": 0.5, "color": COLORS["residential_roads"]},
}

# --------------------------------------------------
# FETCH DATA
# --------------------------------------------------
place_name = "Kathmandu, Nepal"
area_gdf = ox.geocode_to_gdf(place_name)
xmin, ymin, xmax, ymax = area_gdf.total_bounds

roads = ox.features_from_place(place_name, tags={"highway": True})
water = ox.features_from_place(place_name, tags={"natural": "water"})
parks = ox.features_from_place(place_name, tags={"leisure": "park"})
landuse = ox.features_from_place(place_name, tags={"landuse": True, "amenity": True})
buildings = ox.features_from_place(place_name, tags={"building": True})

government_land = landuse[
    (landuse.get("landuse") == "government") |
    (landuse.get("amenity") == "government")
].copy()

admin_buildings = buildings[
    (buildings.get("office") == "government") |
    (buildings.get("amenity").isin(["ministry", "government"])) |
    (buildings.get("building") == "government")
].copy()

# --------------------------------------------------
# CRS SAFETY
# --------------------------------------------------
for gdf in [roads, water, parks, government_land, admin_buildings]:
    if not gdf.empty:
        if gdf.crs is None:
            gdf.set_crs(epsg=4326, inplace=True)
        else:
            gdf.to_crs(epsg=4326, inplace=True)

# --------------------------------------------------
# DISPLAY BOUNDS
# --------------------------------------------------
center_x = (xmin + xmax) / 2
center_y = (ymin + ymax) / 2
half_size = max(xmax - xmin, ymax - ymin) / 2 * 1.1

xmin_disp = center_x - half_size
xmax_disp = center_x + half_size
ymin_disp = center_y - half_size
ymax_disp = center_y + half_size

# --------------------------------------------------
# FIGURE
# --------------------------------------------------
fig, ax = plt.subplots(figsize=(20, 24))
fig.patch.set_facecolor(COLORS["background"])
ax.set_facecolor(COLORS["background"])
ax.set_xlim(xmin_disp, xmax_disp)
ax.set_ylim(ymin_disp, ymax_disp)

# --------------------------------------------------
# BASE LAYERS
# --------------------------------------------------
if not parks.empty:
    parks.plot(ax=ax, color=COLORS["park_green"], alpha=0.6, linewidth=0)
if not water.empty:
    water.plot(ax=ax, color=COLORS["water"], alpha=0.9, linewidth=0)
if not government_land.empty:
    government_land.plot(ax=ax, color=COLORS["government_land"], alpha=0.6, linewidth=0)

# --------------------------------------------------
# ROADS
# --------------------------------------------------
for key in ["residential", "tertiary", "secondary", "primary"]:
    subset = roads[roads["highway"].isin(ROAD_HIERARCHY[key]["types"])]
    if not subset.empty:
        subset.plot(
            ax=ax,
            color=ROAD_HIERARCHY[key]["color"],
            linewidth=ROAD_HIERARCHY[key]["width"],
            alpha=0.9,
            zorder=10
        )

# --------------------------------------------------
# ADMIN BUILDINGS
# --------------------------------------------------
if not admin_buildings.empty:
    admin_buildings.plot(ax=ax, color=COLORS["admin_buildings"], edgecolor="none", zorder=15)

# --------------------------------------------------
# LANDMARKS + LEGEND
# --------------------------------------------------
landmarks = {
    "Swyambhu": (27.7149, 85.2906),
    "Boudhanath " : (27.721422, 85.362024),
    "पशुपतिनाथ ": (27.7105, 85.3486),
    "Narayanhiti Palace Mueseum": (27.714942, 85.319325),
    "Rashtrapati Bhawan": (27.732511, 85.326428),
    "Kathmandu Durbar Square": (27.7043, 85.3080),
    "Singha Durbar": (27.69778, 85.32373),
    "Tribhuwan Intn'l Airport": (27.6966, 85.3590),
    "Dharahara" :(27.700440, 85.312085),
}

landmark_gdf = gpd.GeoDataFrame(
    {"name": list(landmarks.keys())},
    geometry=[Point(lon, lat) for lat, lon in landmarks.values()],
    crs="EPSG:4326"
)

for idx, row in landmark_gdf.iterrows():
    ax.scatter(row.geometry.x, row.geometry.y, color="red", s=120, zorder=25)
    ax.text(
        row.geometry.x, row.geometry.y, str(idx + 1),
        color="white", fontsize=10, fontweight="bold",
        ha="center", va="center", zorder=26
    )

legend_elements = [
    Line2D([0], [0], marker='o', color='w',
           label=f"{i+1}. {name}",
           markerfacecolor='red', markersize=10)
    for i, name in enumerate(landmarks.keys())
]

legend = ax.legend(
    handles=legend_elements,
    loc="lower left",
    bbox_to_anchor=(0.00, 0.11),
    frameon=True,
    framealpha=0.95,
    facecolor="white",
    edgecolor="#cfc8bf",
    borderpad=1.0,
    labelspacing=0.8,
    fontsize=10,
    title="Landmarks",
    title_fontsize=11
)
legend.get_frame().set_zorder(100)

# --------------------------------------------------
# EXTRA OUTLINE IMAGE
# --------------------------------------------------
outline_path = r"E:\QGIS\ne_10m_land\Datasets_contour\30_dayChallenge\07_Urban\results\ktm_outline.png"
outline_img = mpimg.imread(outline_path)
outline_img = simple_black_outline(outline_img, white_threshold=200)

outline_width = 0.95
outline_height = 0.14
x0 = (1.0 - outline_width) / 2
y0 = 0.02

ax.imshow(
    outline_img,
    transform=ax.transAxes,
    extent=(x0, x0 + outline_width, y0, y0 + outline_height),
    interpolation='bicubic',
    aspect='auto',
    zorder=5
)

# --------------------------------------------------
# TITLES
# --------------------------------------------------
title_effect = withStroke(linewidth=3, foreground=COLORS["background"])
ax.text(
    0.5, 0.97, "KATHMANDU",
    ha="center", va="top",
    transform=ax.transAxes,
    fontsize=38,
    fontweight="bold",
    fontfamily="serif",
    color=COLORS["text_primary"],
    path_effects=[title_effect],
    zorder=100
)
ax.text(
    0.5, 0.935, "काठमाडौं महानगर",
    ha="center", va="top",
    transform=ax.transAxes,
    fontsize=14,
    fontfamily="serif",
    color=COLORS["text_secondary"],
    zorder=100
)

# --------------------------------------------------
# NORTH ARROW (INSIDE AXES, UPPER RIGHT)
# --------------------------------------------------
draw_nepali_pagoda_north_arrow(
    ax,
    x=0.92,
    y=0.93,
    size=0.07,
    color=COLORS["text_primary"]
)

# --------------------------------------------------
# CREDIT
# --------------------------------------------------
ax.text(
    0.98, 0.018,
    "@prakash023 • २०८२",
    transform=ax.transAxes,
    ha="right",
    va="bottom",
    fontsize=7,
    fontweight="semibold",
    color="#6b6b6b",
    alpha=0.9,
    rotation= 90,
    zorder=90
)

ax.set_axis_off()

# --------------------------------------------------
# BORDER
# --------------------------------------------------
border = mpatches.Rectangle(
    (xmin_disp, ymin_disp),
    xmax_disp - xmin_disp,
    ymax_disp - ymin_disp,
    linewidth=1.5,
    edgecolor="#cfc8bf",
    facecolor="none",
    zorder=30
)
ax.add_patch(border)

# --------------------------------------------------
# SAVE OUTPUT
# --------------------------------------------------
output_folder = r"E:\QGIS\ne_10m_land\Datasets_contour\30_dayChallenge\07_Urban\results"
os.makedirs(output_folder, exist_ok=True)

plt.savefig(
    os.path.join(output_folder, "Kathmandu_Urban_Minimal_Landmarks_Numbered_StretchedOutline.jpeg"),
    dpi=300,
    bbox_inches="tight",
    facecolor=COLORS["background"]
)

plt.savefig(
    os.path.join(output_folder, "Kathmandu_Urban_Minimal_Landmarks_Numbered_StretchedOutline.svg"),
    bbox_inches="tight",
    facecolor=COLORS["background"]
)

plt.show()
